# California lotto number prediction project

### data load

In [1]:
import pandas as pd

# df = pd.read_csv("lotto.csv", index_col=0) # No order
df = pd.read_csv("lotto_ord.csv", index_col=0) # with order

Use only the most recent n data

In [2]:
train = df#[-200:]

In [3]:
train

,1,2,3,4,5,6,B
1,37,23,10,33,29,40,16
2,42,21,9,25,32,13,2
3,31,21,27,19,11,16,30
4,40,30,14,42,31,27,2
5,16,42,29,40,24,41,3
...,...,...,...,...,...,...,...
997,16,7,4,44,24,14,20
998,18,45,13,20,17,42,41
999,9,1,28,3,18,14,34
1000,19,2,8,22,42,32,39


Process the winning number list to create sequences

In [4]:
sequences = list()
for _, seq in train.iterrows():
    for i in range(1, len(seq)):
        sequence = list(seq)[:i+1]
        sequences.append(sequence)

print('Number of samples to use for training: %d' % len(sequences))

Number of samples to use for training: 6006


In [5]:
train.iloc[0].tolist()

[37, 23, 10, 33, 29, 40, 16]

In [6]:
sequences[:5]

[[37, 23],
 [37, 23, 10],
 [37, 23, 10, 33],
 [37, 23, 10, 33, 29],
 [37, 23, 10, 33, 29, 40]]

잘린 sequences의 길이를 padding

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=7, padding='pre')

In [8]:
import numpy as np

sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
print("X.shape:", X.shape)
print("y.shape:", y.shape)

X.shape: (6006, 6)
y.shape: (6006,)


It is padded with zeros after padding. (0-45 total size of 46)

In [9]:
sequences[:5]

array([[ 0,  0,  0,  0,  0, 37, 23],
       [ 0,  0,  0,  0, 37, 23, 10],
       [ 0,  0,  0, 37, 23, 10, 33],
       [ 0,  0, 37, 23, 10, 33, 29],
       [ 0, 37, 23, 10, 33, 29, 40]])

Separate Test/Verification Datasets

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
X_valid[:3]

array([[ 0,  0,  0,  0,  6, 12],
       [ 0,  0,  0,  0, 35,  5],
       [ 0,  0, 11, 23, 40, 17]])

### create model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Embedding

model = Sequential()
model.add(Embedding(45+1, 10))
model.add(LSTM(64, input_shape=(6, 1), return_sequences=True))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(128, activation='relu'))
model.add(Dense(45+1, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='acc')

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Embedding

model = Sequential()
model.add(Embedding(45+1, 10))
model.add(Bidirectional(LSTM(64, input_shape=(6, 1), return_sequences=True)))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dense(128, activation='relu'))
model.add(Dense(45+1, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='acc')

In [14]:
model.fit(X_train, y_train, batch_size=64, epochs=200, verbose=1, validation_data=(X_valid, y_valid))

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/200
63/63 [==============================] - 7s 28ms/step - loss: 3.8247 - acc: 0.0221 - val_loss: 3.8207 - val_acc: 0.0237
Epoch 2/200
63/63 [==============================] - 1s 13ms/step - loss: 3.8139 - acc: 0.0244 - val_loss: 3.8174 - val_acc: 0.0197
Epoch 3/200
63/63 [==============================] - 1s 13ms/step - loss: 3.8101 - acc: 0.0261 - val_loss: 3.8146 - val_acc: 0.0227
Epoch 4/200
63/63 [==============================] - 1s 13ms/step - loss: 3.8061 - acc: 0.0244 - val_loss: 3.8155 - val_acc: 0.0192
Epoch 5/200
63/63 [==============================] - 1s 13ms/step - loss: 3.8034 - acc: 0.0301 - val_loss: 3.8148 - val_acc: 0.0192
Epoch 6/200
63/63 [==============================] - 1s 13ms/step - loss: 3.7993 - acc: 0.0253 - val_loss: 3.8168 - val_acc: 0.0187
Epoch 7/200
63/63 [==============================] - 1s 13ms/step - lo

Epoch 60/200
63/63 [==============================] - 1s 13ms/step - loss: 1.3535 - acc: 0.6658 - val_loss: 6.4698 - val_acc: 0.0192
Epoch 61/200
63/63 [==============================] - 1s 13ms/step - loss: 1.3133 - acc: 0.6732 - val_loss: 6.5736 - val_acc: 0.0232
Epoch 62/200
63/63 [==============================] - 1s 13ms/step - loss: 1.2783 - acc: 0.6834 - val_loss: 6.7413 - val_acc: 0.0222
Epoch 63/200
63/63 [==============================] - 1s 13ms/step - loss: 1.2415 - acc: 0.6891 - val_loss: 6.7613 - val_acc: 0.0222
Epoch 64/200
63/63 [==============================] - 1s 13ms/step - loss: 1.2138 - acc: 0.6991 - val_loss: 6.8348 - val_acc: 0.0277
Epoch 65/200
63/63 [==============================] - 1s 14ms/step - loss: 1.1875 - acc: 0.7015 - val_loss: 6.9396 - val_acc: 0.0187
Epoch 66/200
63/63 [==============================] - 1s 12ms/step - loss: 1.1522 - acc: 0.7172 - val_loss: 7.0669 - val_acc: 0.0207
Epoch 67/200
63/63 [==============================] - 1s 12ms/step - 

63/63 [==============================] - 1s 13ms/step - loss: 0.6225 - acc: 0.8022 - val_loss: 9.8836 - val_acc: 0.0252
Epoch 122/200
63/63 [==============================] - 1s 13ms/step - loss: 0.6224 - acc: 0.8044 - val_loss: 10.1127 - val_acc: 0.0212
Epoch 123/200
63/63 [==============================] - 1s 13ms/step - loss: 0.6466 - acc: 0.7972 - val_loss: 10.0652 - val_acc: 0.0237
Epoch 124/200
63/63 [==============================] - 1s 13ms/step - loss: 0.6377 - acc: 0.7970 - val_loss: 10.1400 - val_acc: 0.0172
Epoch 125/200
63/63 [==============================] - 1s 13ms/step - loss: 0.6155 - acc: 0.8007 - val_loss: 10.2128 - val_acc: 0.0207
Epoch 126/200
63/63 [==============================] - 1s 15ms/step - loss: 0.6038 - acc: 0.8049 - val_loss: 10.1796 - val_acc: 0.0242
Epoch 127/200
63/63 [==============================] - 1s 13ms/step - loss: 0.6052 - acc: 0.8042 - val_loss: 10.1742 - val_acc: 0.0212
Epoch 128/200
63/63 [==============================] - 1s 13ms/step - 

Epoch 182/200
63/63 [==============================] - 1s 13ms/step - loss: 0.5710 - acc: 0.8022 - val_loss: 11.0344 - val_acc: 0.0232
Epoch 183/200
63/63 [==============================] - 1s 14ms/step - loss: 0.5671 - acc: 0.8064 - val_loss: 11.0845 - val_acc: 0.0182
Epoch 184/200
63/63 [==============================] - 1s 12ms/step - loss: 0.5725 - acc: 0.8029 - val_loss: 11.0347 - val_acc: 0.0202
Epoch 185/200
63/63 [==============================] - 1s 13ms/step - loss: 0.5739 - acc: 0.8059 - val_loss: 11.1054 - val_acc: 0.0207
Epoch 186/200
63/63 [==============================] - 1s 12ms/step - loss: 0.5739 - acc: 0.8027 - val_loss: 11.0401 - val_acc: 0.0192
Epoch 187/200
63/63 [==============================] - 1s 14ms/step - loss: 0.5721 - acc: 0.8039 - val_loss: 10.9535 - val_acc: 0.0212
Epoch 188/200
63/63 [==============================] - 1s 16ms/step - loss: 0.5712 - acc: 0.8047 - val_loss: 11.1239 - val_acc: 0.0192
Epoch 189/200
63/63 [==============================] - 

### function returning a list of numbers

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def seq_generate(model, input_num, verbose=False): 
    sequence = [input_num]

    while len(sequence) < 7:
        encoded = pad_sequences([sequence], maxlen=7, padding='pre')
        result = model.predict(encoded, verbose=0)
        
        mask = np.zeros(result.size, dtype=bool)
        mask[sequence] = True
        result = np.ma.array(result, mask=mask)
        result = np.argmax(result)

        sequence.append(result)
        
        if verbose:
            print("sequence:", sequence)

    return [sorted(sequence[:-1]), sequence[-1]]

테스트

In [16]:
test_num = 2
seq = [[test_num]]
encoded = pad_sequences(seq, maxlen=7, padding='pre')
result = model.predict(encoded, verbose=0)

print(result)
print(np.argmax(result))

seq_generate(model, test_num, verbose=True)

[[1.0314834e-17 8.9763565e-04 3.1238937e-07 4.7520551e-08 1.3271838e-05
  7.9703137e-02 2.2938398e-01 1.2309606e-03 1.2171864e-01 9.8727259e-04
  2.7534894e-05 5.7089310e-02 5.0864004e-02 8.9254837e-07 3.9849801e-05
  9.2154165e-04 5.5561084e-02 5.2318823e-02 4.5428376e-07 8.0502074e-04
  5.8052409e-04 7.8816875e-04 8.4111176e-02 4.8434456e-05 4.7312543e-02
  1.4438300e-03 6.1117564e-05 3.3328225e-04 5.8724955e-02 3.4168109e-02
  1.8935837e-02 5.4489370e-03 3.2386813e-02 6.5905544e-05 2.8633702e-05
  6.1230071e-02 1.3459387e-05 4.4922122e-05 2.3244693e-05 4.1275352e-04
  2.1671376e-05 2.4509643e-07 3.5191953e-04 4.8888690e-04 1.3760059e-03
  3.4814140e-05]]
6
sequence: [2, 6]
sequence: [2, 6, 13]
sequence: [2, 6, 13, 27]
sequence: [2, 6, 13, 27, 43]
sequence: [2, 6, 13, 27, 43, 17]
sequence: [2, 6, 13, 27, 43, 17, 23]


[[2, 6, 13, 17, 27, 43], 23]

### performance result

Print the first number in order of frequency

In [17]:
first_num = df['1'].value_counts()[:6]
first_num

37    35
25    31
26    31
23    29
4     28
45    28
Name: 1, dtype: int64

In [18]:
for num, _ in first_num.iteritems():
    print(f"{num:2}:", seq_generate(model, num))

37: [[10, 18, 20, 31, 37, 42], 27]
25: [[6, 15, 19, 25, 33, 38], 26]
26: [[5, 20, 26, 27, 35, 45], 23]
23: [[3, 4, 22, 23, 36, 41], 9]
 4: [[3, 4, 16, 37, 38, 40], 30]
45: [[1, 13, 20, 26, 29, 45], 39]


모든 숫자에 대해 결과 출력

In [19]:
for i in range(1, 46):
    print(f"{i:2}:", seq_generate(model, i))

 1: [[1, 3, 12, 13, 24, 44], 31]
 2: [[2, 6, 13, 17, 27, 43], 23]
 3: [[3, 8, 21, 22, 34, 41], 12]
 4: [[3, 4, 16, 37, 38, 40], 30]
 5: [[4, 5, 9, 15, 26, 27], 42]
 6: [[2, 4, 6, 10, 11, 37], 28]
 7: [[1, 2, 7, 15, 23, 34], 17]
 8: [[8, 10, 14, 31, 34, 36], 12]
 9: [[6, 9, 12, 14, 26, 37], 31]
10: [[5, 10, 13, 21, 28, 35], 9]
11: [[10, 11, 18, 22, 28, 39], 30]
12: [[1, 10, 12, 18, 24, 35], 31]
13: [[6, 13, 25, 28, 38, 45], 39]
14: [[8, 14, 18, 30, 31, 44], 15]
15: [[6, 10, 11, 15, 19, 34], 25]
16: [[16, 17, 23, 24, 29, 44], 3]
17: [[4, 5, 6, 8, 17, 39], 25]
18: [[13, 15, 17, 18, 25, 33], 37]
19: [[6, 11, 16, 19, 20, 28], 12]
20: [[3, 20, 22, 33, 40, 44], 8]
21: [[11, 15, 21, 22, 26, 35], 37]
22: [[9, 14, 20, 22, 33, 34], 13]
23: [[3, 4, 22, 23, 36, 41], 9]
24: [[8, 15, 24, 26, 39, 41], 22]
25: [[6, 15, 19, 25, 33, 38], 26]
26: [[5, 20, 26, 27, 35, 45], 23]
27: [[2, 13, 21, 25, 27, 42], 38]
28: [[5, 18, 19, 28, 31, 32], 12]
29: [[7, 8, 11, 29, 32, 34], 20]
30: [[10, 12, 13, 26, 29, 30],